## Vova Bravo

### Hutchinson estimator of the trace

In [1]:
import numpy as np
from numba import njit
import cProfile
import time
import cupy as cp

In [2]:
@njit
def t_comp(eps, delta):
    fig = 8./(eps**2) * np.log(2./delta)
    fig = int(fig+1)
    return fig

In [3]:
@njit
def c(p):
    fig = (2 + int(p))**(p+1)
    if p > 1:
        array = np.array([abs((p - i + 1)/i) for i in range(1, int(p) + 1)])
        nafig = np.prod(array)
    else:
        nafig = 1
    return fig*nafig

In [4]:
@njit
def m_comp(c, p, eps, n):
    fig = 7* np.power(3*c*n/(p*eps), 1/p)
    return np.int(fig + 1)

In [5]:
@njit
def beta(m, c, p):
    fig = (c/p) * np.power(float(m)+1, -p)
    return fig

In [22]:
@njit
def hutch(X, p, m, t):
    y = 0
    n = X.shape[0]
    for i in range(1,t+1):
        g_i = np.random.binomial(1, 0.5, size=(n,))  * 2.0 - 1.0
        v_k = X@g_i
        u_k = g_i@v_k
        a_k = p
        S_i_k = a_k*u_k
        for k in range(2, m+1):
            v_k = X@v_k
            u_k = g_i@v_k
            a_k = a_k * (p-(k-1))/k
            if np.abs(a_k) < 1e-8:
                break
            S_i_k = S_i_k + (((-1)**(k-1)) * a_k) * u_k
        y += S_i_k
    y = y / t
    return y

In [23]:
@njit
def power_method(A, x0, num_iter):
    for iter in range(num_iter):
        x0 = A @ x0
        x0 = x0 / np.linalg.norm(x0)
        approx = (A @ x0)
        l = x0 @ approx
    return x0, l

# def power_method_vec(A, X, num_iter):
#     xp = cp.get_array_module(A)
#     for iter in range(num_iter):
#         X = A @ X
#         X = X / xp.linalg.norm(X, axis=0)
#         approx = (A @ X)
#         ls =  np.sum(X * approx, axis=0)
#     return ls

In [24]:
@njit
def alpha(A, delta):
    n = A.shape[0]
    q = int(4.82 * np.log(1. / delta) + 1)
    t = int(0.5 * np.log(4 * n) + 1)
    max_lambda = 0
    for i in range(q):
        x0 = np.random.binomial(1, 0.5, size=(n,)) * 2.0 - 1.0
        x, l = power_method(A, x0, t)
        if l > max_lambda:
            max_lambda = l
    return max_lambda

In [25]:
@njit
def vova_bravo_without(A, p, eps, delta):
    n = A.shape[0]
    t = t_comp(eps, delta)
    c_p = c(p)
    m = m_comp(c_p, p, eps, n)
    b_m = beta(m, c_p, p)
    a = alpha(A, delta)
    print(t, m)
    return np.power(a, p) * np.floor((1+b_m)*n - hutch(np.eye(n) - A / a, p, m, t))

In [26]:
def compute_true_schatten(A, p):
    u, s, vh = np.linalg.svd(A)
    return np.power(s, p).sum()

In [31]:
A = np.random.normal(size=(200, 200))
A = A.T @ A

In [36]:
p = 5
start = time.time()
schatten_vova_bravo = vova_bravo_without(A, p, eps=0.1, delta=0.05)
print(time.time() - start)

2952 299
0.9783480167388916


In [33]:
schatten_true = compute_true_schatten(A, p)

## Error computation

In [34]:
np.abs((schatten_vova_bravo - schatten_true) / schatten_true)

0.03940280358506373

## Non-positive definite

In [14]:
import scipy.special

In [15]:
def h_tilde(A, m, p):
    A_k = A
    h_tilde = np.zeros_like(A)
    for i in range(1, m+1):
        h_tilde += (-1 if i % 2 == 0 else 1) * scipy.special.binom(p, i) * A_k
        A_k = A_k @ A
    return h_tilde

In [16]:
p = 5
eps = 0.05
delta = 0.01
n = A.shape[0]
t = t_comp(eps, delta)
c_p = c(p)
m = m_comp(c_p, p, eps, n)
b_m = beta(m, c_p, p)
a = 6 * alpha(A, delta)

In [ ]:
hutch(np.eye(n) - A / a, p, m, t)

In [ ]:
def hutch_test(A, eps, delta):
    p = int(20*np.log(2/delta)/(eps)**2)
    gamma = 0
    for i in range(p):
        g = np.random.randn(A.shape[0])
        fig = A.dot(g)
        nafig = g.dot(fig)
        gamma = gamma + nafig
    gamma = gamma/p
    return gamma

In [ ]:
h_tilde(np.eye(n) - A / a, m, p).trace()

In [ ]:
hutch_test(h_tilde(np.eye(n) - A / a, m, p), eps, delta)